## Process expression and covariate data
Save V8 covariates to HDF5 format, for use with downstream analysis.

For covariate data we also compute a version of its orthonormal basis to be used for `mr-ash` analysis.

In [ ]:
[global]
cwd = "~/Documents/GTExV8"

In [ ]:
[format_covariates]
# Consolidate covariates to HDF5 format
input: glob.glob("${cwd!a}/GTEx_Analysis_v8_eQTL_covariates/*.v8.covariates.txt")
output: "${cwd!a}/GTExV8.covariates.h5", "${cwd!a}/GTExV8.covariates.orth.h5"
task: workdir = cwd
python:
    for item in [${input!ar,}]:
        peer = pd.read_csv(item, header = 0, sep = '\t', index_col = 0).transpose()
        samples = {}
        for x in peer.index:
            samples[x] = dat.loc['-'.join(x.split('-')[:2])].tolist()
        samples = pd.DataFrame(samples).transpose()
        samples.columns = dat.columns
        samples = samples.merge(peer, left_index = True, right_index = True)
        samples.to_hdf(${output[0]!ar}, '/{}'.format(os.path.basename(item[:-20])), mode = 'a', complevel = 9, complib = 'zlib')
        samples_orth = pd.DataFrame(scipy.linalg.orth(samples), index = samples.index)
        samples_orth.to_hdf(${output[1]!ar}, '/{}'.format(os.path.basename(item[:-20])), mode = 'a', complevel = 9, complib = 'zlib')